In [41]:
import pandas as pd
import numpy as np

# 1. Đọc file CSV với pandas (khuyến nghị)
df = pd.read_csv('/home/duy/Downloads/Mallorn_update/Mallorn/mallorn-astronomical-classification-challenge/train_features_ml.csv')
print("Kích thước dữ liệu:", df.shape)
print("\n5 dòng đầu:")
print(df.head())

print("\nThông tin cơ bản:")
print(df.info())

Kích thước dữ liệu: (3043, 117)

5 dòng đầu:
                  object_id       Z  Z_err    EBV  target  u_n_obs  \
0  Dornhoth_fervain_onodrim  3.0490    NaN  0.110       0        5   
1       Dornhoth_galadh_ylf  0.4324    NaN  0.058       0       15   
2      Elrim_melethril_thul  0.4673    NaN  0.577       0        5   
3        Ithil_tobas_rodwen  0.6946    NaN  0.012       0      108   
4       Mirion_adar_Druadan  0.4161    NaN  0.058       0       13   

   u_time_span  u_flux_mean  u_flux_std  u_flux_min  ...  color_mean_g_i  \
0     849.3841     0.584015    1.022628    0.000435  ...       -2.485906   
1    2246.6157     0.027015    0.545088   -0.799254  ...       -0.173395   
2     498.8939     0.006054    0.293108   -0.427984  ...       -2.060860   
3    2858.4129     0.152980    0.439240   -1.455301  ...       -0.167220   
4    1859.9219    -0.014067    0.406413   -0.828860  ...       -0.337740   

   color_mean_r_z  t_peak_diff_u_g  t_peak_diff_g_r  t_peak_diff_r_i  \
0    

In [42]:
# ============================================================
# CHECK VÀ XỬ LÝ CÁC GIÁ TRỊ NaN
# ============================================================

print("="*60)
print("TRƯỚC KHI XỬ LÝ NaN")
print("="*60)
print(f"Kích thước DataFrame: {df.shape}")
print(f"\nTổng số NaN trong mỗi cột:")
nan_counts = df.isnull().sum()
print(nan_counts[nan_counts > 0])

print(f"\nSố hàng có ít nhất 1 NaN: {df.isnull().any(axis=1).sum()}")
print(f"Tổng số NaN trong toàn bộ DataFrame: {df.isnull().sum().sum()}")

# ============================================================
# ĐIỀN GIÁ TRỊ 0 VÀO CỘT Z_err
# ============================================================

print("\n" + "="*60)
print("XỬ LÝ NaN")
print("="*60)

# Điền 0 vào cột Z_err
print("1. Điền giá trị 0 vào cột Z_err...")
df['Z_err'] = df['Z_err'].fillna(0)
print(f"   ✓ Đã điền 0 vào {nan_counts['Z_err']} giá trị NaN trong Z_err")

# Xóa các hàng có NaN ở các cột khác (không phải Z_err)
print("\n2. Xóa các hàng có NaN ở các cột khác...")
df_cleaned = df.dropna()

print("\n" + "="*60)
print("SAU KHI XỬ LÝ NaN")
print("="*60)
print(f"Kích thước DataFrame sau khi xử lý: {df_cleaned.shape}")
print(f"Số hàng đã xóa: {len(df) - len(df_cleaned)}")
print(f"% hàng đã xóa: {((len(df) - len(df_cleaned)) / len(df) * 100):.2f}%")

# Kiểm tra xem còn NaN không
print(f"\nCòn NaN trong DataFrame không? {df_cleaned.isnull().any().any()}")
print(f"Tổng số NaN còn lại: {df_cleaned.isnull().sum().sum()}")

# Hiển thị thống kê cột Z_err sau khi điền
print(f"\nThống kê cột Z_err sau khi điền 0:")
print(f"  - Min: {df_cleaned['Z_err'].min()}")
print(f"  - Max: {df_cleaned['Z_err'].max()}")
print(f"  - Mean: {df_cleaned['Z_err'].mean():.4f}")
print(f"  - Số giá trị = 0: {(df_cleaned['Z_err'] == 0).sum()}")

# Cập nhật df với bản đã clean
df = df_cleaned.copy()

print("\n✓ DataFrame đã được cập nhật!")
print(f"✓ Kích thước hiện tại: {df.shape}")


TRƯỚC KHI XỬ LÝ NaN
Kích thước DataFrame: (3043, 117)

Tổng số NaN trong mỗi cột:
Z_err              3043
u_time_span          16
u_flux_mean          41
u_flux_std           41
u_flux_min           41
                   ... 
t_peak_diff_u_g      30
t_peak_diff_g_r       2
t_peak_diff_z_y      19
t_peak_diff_u_r      28
t_peak_diff_g_i       2
Length: 92, dtype: int64

Số hàng có ít nhất 1 NaN: 3043
Tổng số NaN trong toàn bộ DataFrame: 6925

XỬ LÝ NaN
1. Điền giá trị 0 vào cột Z_err...
   ✓ Đã điền 0 vào 3043 giá trị NaN trong Z_err

2. Xóa các hàng có NaN ở các cột khác...

SAU KHI XỬ LÝ NaN
Kích thước DataFrame sau khi xử lý: (2876, 117)
Số hàng đã xóa: 167
% hàng đã xóa: 5.49%

Còn NaN trong DataFrame không? False
Tổng số NaN còn lại: 0

Thống kê cột Z_err sau khi điền 0:
  - Min: 0.0
  - Max: 0.0
  - Mean: 0.0000
  - Số giá trị = 0: 2876

✓ DataFrame đã được cập nhật!
✓ Kích thước hiện tại: (2876, 117)


In [43]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Tạo bản sao của DataFrame để không ảnh hưởng đến dữ liệu gốc
df_scaled = df.copy()

# Xác định các cột cần scaling (bỏ qua cột tên bệnh nhân và label)
columns_to_scale = [col for col in df_scaled.columns 
                   if col not in ['object_id', 'Z_err', 'target']]

print("Các cột sẽ được scaling:")
print(columns_to_scale[:10], "...")  # Hiển thị 10 cột đầu
print(f"Tổng số cột cần scaling: {len(columns_to_scale)}")

# Hiển thị thống kê trước khi scaling
print("\nThống kê trước khi scaling (5 cột đầu):")
print(df_scaled[columns_to_scale[:5]].describe())

Các cột sẽ được scaling:
['Z', 'EBV', 'u_n_obs', 'u_time_span', 'u_flux_mean', 'u_flux_std', 'u_flux_min', 'u_flux_max', 'u_flux_median', 'u_amplitude'] ...
Tổng số cột cần scaling: 114

Thống kê trước khi scaling (5 cột đầu):
                 Z          EBV      u_n_obs  u_time_span  u_flux_mean
count  2876.000000  2876.000000  2876.000000  2876.000000  2876.000000
mean      0.682277     0.053392    13.598053  1852.156176     0.393870
std       0.542379     0.054949    15.015810   557.017193     1.590099
min       0.008771     0.002000     3.000000    28.247300   -17.135965
25%       0.324500     0.021000     9.000000  1481.598725     0.002790
50%       0.493800     0.037000    12.000000  1863.958150     0.137668
75%       0.896825     0.067000    14.000000  2219.633650     0.372800
max       4.924000     0.758000   161.000000  3525.770100    50.071001


In [44]:
df_scaled

,object_id,Z,Z_err,EBV,target,u_n_obs,u_time_span,u_flux_mean,u_flux_std,u_flux_min,...,color_mean_g_i,color_mean_r_z,t_peak_diff_u_g,t_peak_diff_g_r,t_peak_diff_r_i,t_peak_diff_i_z,t_peak_diff_z_y,t_peak_diff_u_r,t_peak_diff_g_i,t_peak_diff_r_z
0,Dornhoth_fervain_onodrim,3.0490,0.0,0.110,0,5,849.3841,0.584015,1.022628,0.000435,...,-2.485906,-0.194878,-382.8829,44.0095,-44.0096,-13.2029,-17.6038,-338.8734,-0.0001,-57.2125
1,Dornhoth_galadh_ylf,0.4324,0.0,0.058,0,15,2246.6157,0.027015,0.545088,-0.799254,...,-0.173395,-0.252129,-821.6194,51.3512,-20.5406,-64.1890,-20.5404,-770.2682,30.8106,-84.7296
3,Ithil_tobas_rodwen,0.6946,0.0,0.012,0,108,2858.4129,0.152980,0.439240,-1.455301,...,-0.167220,-0.096281,1661.2019,-1331.8257,1331.8257,25.7773,-1632.5606,329.3762,0.0000,1357.6030
4,Mirion_adar_Druadan,0.4161,0.0,0.058,0,13,1859.9219,-0.014067,0.406413,-0.828860,...,-0.337740,-0.086716,-48.5965,-234.1469,-108.2378,474.9207,-523.5172,-282.7434,-342.3847,366.6829
5,Mirion_lalaith_neledh,1.1970,0.0,0.054,0,8,1832.8097,0.422299,0.288031,-0.023856,...,-0.394888,-0.449806,-99.0709,1183.0219,-1177.1941,1369.5080,-1427.7850,1083.9510,5.8278,192.3139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3038,tinnu_gellui_tathar,0.8898,0.0,0.042,0,16,2189.5612,-0.068665,0.395604,-0.825168,...,-0.309896,-0.412576,2085.4235,-595.4538,283.0408,-197.5945,-1746.3085,1489.9697,-312.4130,85.4463
3039,uir_heleg_corf,0.9598,0.0,0.042,0,13,2887.8277,1.134710,2.021232,-2.843739,...,0.342609,0.074489,0.0000,-12.5968,25.1937,176.3559,-289.7275,-12.5968,12.5969,201.5496
3040,uir_rhosc_law,0.1543,0.0,0.024,0,16,1829.4130,0.041323,0.481292,-0.979264,...,-0.361688,-0.500247,60.7013,-4.1863,-35.5835,0.0000,-2.0932,56.5150,-39.7698,-35.5835
3041,uruk_in_pess,1.1520,0.0,0.019,0,13,2568.6786,0.703543,1.046953,-2.024119,...,0.360625,-0.177830,-803.5525,732.9475,-396.7331,245.4366,581.6510,-70.6050,336.2144,-151.2965


In [45]:
# Kiểm tra xem có NaN không
print("Có NaN trong df không?", df.isnull().any().any())

# Đếm số NaN trong mỗi cột
print("\nSố NaN trong mỗi cột:")
print(df.isnull().sum())

# Tính % NaN trong mỗi cột
print("\n% NaN trong mỗi cột:")
print((df.isnull().sum() / len(df)) * 100)

Có NaN trong df không? False

Số NaN trong mỗi cột:
object_id          0
Z                  0
Z_err              0
EBV                0
target             0
                  ..
t_peak_diff_i_z    0
t_peak_diff_z_y    0
t_peak_diff_u_r    0
t_peak_diff_g_i    0
t_peak_diff_r_z    0
Length: 117, dtype: int64

% NaN trong mỗi cột:
object_id          0.0
Z                  0.0
Z_err              0.0
EBV                0.0
target             0.0
                  ... 
t_peak_diff_i_z    0.0
t_peak_diff_z_y    0.0
t_peak_diff_u_r    0.0
t_peak_diff_g_i    0.0
t_peak_diff_r_z    0.0
Length: 117, dtype: float64


In [46]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
scaler = MinMaxScaler()

# Scaling các cột đã chọn
df_scaled[columns_to_scale] = scaler.fit_transform(df_scaled[columns_to_scale])

print("Min-Max Scaling đã hoàn thành!")
print("\nThống kê sau khi scaling (5 cột đầu):")
print(df_scaled[columns_to_scale[:5]].describe())

# Kiểm tra phạm vi giá trị sau scaling
print(f"\nPhạm vi giá trị sau scaling:")
print(f"Min: {df_scaled[columns_to_scale].min().min():.6f}")
print(f"Max: {df_scaled[columns_to_scale].max().max():.6f}")

Min-Max Scaling đã hoàn thành!

Thống kê sau khi scaling (5 cột đầu):
                 Z          EBV      u_n_obs  u_time_span  u_flux_mean
count  2876.000000  2876.000000  2876.000000  2876.000000  2876.000000
mean      0.137024     0.067979     0.067076     0.521486     0.260834
std       0.110347     0.072683     0.095037     0.159260     0.023660
min       0.000000     0.000000     0.000000     0.000000     0.000000
25%       0.064235     0.025132     0.037975     0.415537     0.255015
50%       0.098679     0.046296     0.056962     0.524860     0.257021
75%       0.180674     0.085979     0.069620     0.626554     0.260520
max       1.000000     1.000000     1.000000     1.000000     1.000000

Phạm vi giá trị sau scaling:
Min: 0.000000
Max: 1.000000


In [47]:
df_scaled

,object_id,Z,Z_err,EBV,target,u_n_obs,u_time_span,u_flux_mean,u_flux_std,u_flux_min,...,color_mean_g_i,color_mean_r_z,t_peak_diff_u_g,t_peak_diff_g_r,t_peak_diff_r_i,t_peak_diff_i_z,t_peak_diff_z_y,t_peak_diff_u_r,t_peak_diff_g_i,t_peak_diff_r_z
0,Dornhoth_fervain_onodrim,0.618533,0.0,0.142857,0,0.012658,0.234777,0.263663,0.025473,0.701691,...,0.220216,0.430231,0.461601,0.625388,0.489657,0.481269,0.508584,0.501736,0.515684,0.472637
1,Dornhoth_galadh_ylf,0.086187,0.0,0.074074,0,0.075949,0.634268,0.255375,0.013209,0.684591,...,0.325172,0.426179,0.378973,0.626645,0.494262,0.471119,0.507978,0.415584,0.521944,0.467464
3,Ithil_tobas_rodwen,0.139531,0.0,0.013228,0,0.664557,0.809191,0.257249,0.010491,0.670563,...,0.325453,0.437208,0.846568,0.389680,0.759607,0.489029,0.175839,0.635187,0.515684,0.738604
4,Mirion_adar_Druadan,0.082871,0.0,0.074074,0,0.063291,0.523706,0.254764,0.009648,0.683958,...,0.317713,0.437885,0.524558,0.577734,0.477055,0.578441,0.404345,0.512945,0.446111,0.552323
5,Mirion_lalaith_neledh,0.241744,0.0,0.068783,0,0.031646,0.515954,0.261257,0.006607,0.701172,...,0.315120,0.412191,0.515052,0.820523,0.267317,0.756529,0.218031,0.785879,0.516868,0.519544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3038,tinnu_gellui_tathar,0.179245,0.0,0.052910,0,0.082278,0.617956,0.253951,0.009370,0.684037,...,0.318977,0.414826,0.926462,0.515835,0.553827,0.444562,0.152402,0.866962,0.452201,0.499455
3039,uir_heleg_corf,0.193486,0.0,0.052910,0,0.063291,0.817602,0.271857,0.051119,0.640874,...,0.348592,0.449292,0.533710,0.615690,0.503236,0.519005,0.452515,0.566894,0.518243,0.521281
3040,uir_rhosc_law,0.029608,0.0,0.029101,0,0.082278,0.514983,0.255588,0.011571,0.680742,...,0.316626,0.408622,0.545142,0.617131,0.491311,0.483898,0.511779,0.580696,0.507602,0.476703
3041,uruk_in_pess,0.232589,0.0,0.022487,0,0.063291,0.726352,0.265441,0.026098,0.658400,...,0.349410,0.431437,0.382375,0.743416,0.420450,0.532757,0.632054,0.555310,0.584002,0.454950


In [49]:
X_base = df_scaled.copy()
X_base.drop(['Z_err'], axis=1, inplace=True)
y_base = X_base['target'].copy()

# KIỂM TRA y_base
print("=" * 60)
print("CHECKING y_base")
print("=" * 60)
print(f"y_base type: {type(y_base)}")
print(f"y_base shape: {y_base.shape}")
print(f"y_base ndim: {y_base.ndim}")
print(f"First 10 values:\n{y_base.head(10)}")
print(f"Value counts:\n{y_base.value_counts()}")

exclude_cols = ['object_id', 'Z_err', 'target']
feature_cols_base = [col for col in X_base.columns 
                     if col not in exclude_cols 
                     and X_base[col].dtype in ['float64', 'int64']]

X_base_features = X_base[feature_cols_base].copy()
print(f"\nX_base_features shape: {X_base_features.shape}")
# X_base_features

CHECKING y_base
y_base type: <class 'pandas.core.series.Series'>
y_base shape: (2876,)
y_base ndim: 1
First 10 values:
0     0
1     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
Name: target, dtype: int64
Value counts:
target
0    2729
1     147
Name: count, dtype: int64

X_base_features shape: (2876, 114)


In [50]:
rows_with_nan = X_base.isnull().any(axis=1)
print("Số hàng có NaN:", rows_with_nan.sum())

# Lấy ra các hàng có NaN
df_with_nan = df[rows_with_nan]
print("\nCác hàng có NaN:")
print(df_with_nan)

# Lấy index của các hàng có NaN
nan_indices = df[rows_with_nan].index.tolist()
print("\nIndex của các hàng có NaN:", nan_indices[:10]) 

Số hàng có NaN: 0

Các hàng có NaN:
Empty DataFrame
Columns: [object_id, Z, Z_err, EBV, target, u_n_obs, u_time_span, u_flux_mean, u_flux_std, u_flux_min, u_flux_max, u_flux_median, u_amplitude, u_flux_skew, u_flux_kurtosis, u_t_peak, u_rise_time, u_decay_time, u_max_slope_up, u_max_slope_down, u_auc, g_n_obs, g_time_span, g_flux_mean, g_flux_std, g_flux_min, g_flux_max, g_flux_median, g_amplitude, g_flux_skew, g_flux_kurtosis, g_t_peak, g_rise_time, g_decay_time, g_max_slope_up, g_max_slope_down, g_auc, r_n_obs, r_time_span, r_flux_mean, r_flux_std, r_flux_min, r_flux_max, r_flux_median, r_amplitude, r_flux_skew, r_flux_kurtosis, r_t_peak, r_rise_time, r_decay_time, r_max_slope_up, r_max_slope_down, r_auc, i_n_obs, i_time_span, i_flux_mean, i_flux_std, i_flux_min, i_flux_max, i_flux_median, i_amplitude, i_flux_skew, i_flux_kurtosis, i_t_peak, i_rise_time, i_decay_time, i_max_slope_up, i_max_slope_down, i_auc, z_n_obs, z_time_span, z_flux_mean, z_flux_std, z_flux_min, z_flux_max, z_flu

In [51]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import (accuracy_score, recall_score, precision_score, 
                             f1_score, roc_auc_score, confusion_matrix,
                             classification_report, roc_curve)
from sklearn.base import clone
import warnings
warnings.filterwarnings('ignore')

# Import models
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                              BaggingClassifier, AdaBoostClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [52]:
def calculate_specificity(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    num_classes = cm.shape[0]
    specificity_list = []

    for i in range(num_classes):
        # TN = tổng tất cả phần tử ngoại trừ hàng i và cột i
        tn = np.sum(np.delete(np.delete(cm, i, axis=0), i, axis=1))
        # FP = tổng cột i (ngoại trừ phần tử đúng)
        fp = np.sum(cm[:, i]) - cm[i, i]
        specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
        specificity_list.append(specificity)

    # trung bình macro (mỗi lớp có trọng số bằng nhau)
    return np.mean(specificity_list)

def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    specificity = calculate_specificity(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='macro')
    precision = precision_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    
    if y_pred_proba is not None:
        # Binary classification: chỉ lấy xác suất của class dương (class 1)
        if y_pred_proba.shape[1] == 2:
            auc_roc = roc_auc_score(y_test, y_pred_proba[:, 1])
        else:
            # Multi-class classification
            auc_roc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='macro')
    else:
        auc_roc = np.nan
        
    results = {
        'Model': model_name,
        'Accuracy': accuracy,
        'Specificity': specificity,
        'Recall': recall,
        'Precision': precision,
        'F1 Score': f1,
        'AUC': auc_roc
    }
    return results, y_pred, y_pred_proba

def cross_validate_model(model, X, y, cv=5, use_stratified=True):
    """
    Thực hiện k-fold cross-validation với đầy đủ metrics
    
    Parameters:
    -----------
    model : sklearn model
        Model cần đánh giá
    X : array-like
        Features
    y : array-like
        Labels
    cv : int
        Số lượng folds (mặc định 5)
    use_stratified : bool
        Sử dụng StratifiedKFold để đảm bảo phân phối lớp (mặc định True)
    """
    from sklearn.model_selection import StratifiedKFold
    
    if use_stratified:
        kfold = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    else:
        kfold = KFold(n_splits=cv, shuffle=True, random_state=42)
    
    # Tính toán các metrics cho từng fold
    fold_accuracies = []
    fold_recalls = []
    fold_precisions = []
    fold_f1s = []
    fold_specificities = []
    fold_aucs = []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y)):
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        
        # Train model
        model_copy = clone(model)
        model_copy.fit(X_train_fold, y_train_fold)
        
        # Predict
        y_pred_fold = model_copy.predict(X_val_fold)
        y_pred_proba_fold = model_copy.predict_proba(X_val_fold)
        
        # Calculate metrics
        fold_accuracies.append(accuracy_score(y_val_fold, y_pred_fold))
        fold_recalls.append(recall_score(y_val_fold, y_pred_fold, average='macro', zero_division=0))
        fold_precisions.append(precision_score(y_val_fold, y_pred_fold, average='macro', zero_division=0))
        fold_f1s.append(f1_score(y_val_fold, y_pred_fold, average='macro', zero_division=0))
        fold_specificities.append(calculate_specificity(y_val_fold, y_pred_fold))
        
        try:
            # Binary classification: chỉ lấy xác suất của class dương (class 1)
            if y_pred_proba_fold.shape[1] == 2:
                fold_aucs.append(roc_auc_score(y_val_fold, y_pred_proba_fold[:, 1]))
            else:
                # Multi-class classification
                fold_aucs.append(roc_auc_score(y_val_fold, y_pred_proba_fold, multi_class='ovr', average='macro'))
        except:
            fold_aucs.append(np.nan)
    
    # Tính mean và std cho tất cả metrics
    cv_results = {
        'CV_Accuracy_mean': np.mean(fold_accuracies),
        'CV_Accuracy_std': np.std(fold_accuracies),
        'CV_Recall_mean': np.mean(fold_recalls),
        'CV_Recall_std': np.std(fold_recalls),
        'CV_Precision_mean': np.mean(fold_precisions),
        'CV_Precision_std': np.std(fold_precisions),
        'CV_F1_mean': np.mean(fold_f1s),
        'CV_F1_std': np.std(fold_f1s),
        'CV_Specificity_mean': np.mean(fold_specificities),
        'CV_Specificity_std': np.std(fold_specificities),
        'CV_AUC_mean': np.nanmean(fold_aucs),
        'CV_AUC_std': np.nanstd(fold_aucs),
        'CV_Fold_Details': {
            'accuracies': fold_accuracies,
            'recalls': fold_recalls,
            'precisions': fold_precisions,
            'f1s': fold_f1s,
            'specificities': fold_specificities,
            'aucs': fold_aucs
        }
    }
    return cv_results


In [53]:
print("\n" + "="*60)
print("Train-Test Split (80:20)")
print("="*60)

# KIỂM TRA y_base trước khi split
print(f"y_base type: {type(y_base)}")
print(f"y_base shape: {y_base.shape}")
print(f"y_base dtype: {y_base.dtype}")
print(f"y_base first 5 values:\n{y_base.head()}")

X_train, X_test, y_train, y_test = train_test_split(
    X_base_features, y_base, 
    test_size=0.2, 
    random_state=42,
    stratify=y_base
)

# KIỂM TRA sau khi split
print(f"\ny_train type: {type(y_train)}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test type: {type(y_test)}")
print(f"y_test shape: {y_test.shape}")

print(f"\nTrain set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"Train set size: {len(X_train)} samples ({len(X_train)/len(X_base_features)*100:.1f}%)")
print(f"Test set size: {len(X_test)} samples ({len(X_test)/len(X_base_features)*100:.1f}%)")
    


Train-Test Split (80:20)
y_base type: <class 'pandas.core.series.Series'>
y_base shape: (2876,)
y_base dtype: int64
y_base first 5 values:
0    0
1    0
3    0
4    0
5    0
Name: target, dtype: int64

y_train type: <class 'pandas.core.series.Series'>
y_train shape: (2300,)
y_test type: <class 'pandas.core.series.Series'>
y_test shape: (576,)

Train set shape: (2300, 114)
Test set shape: (576, 114)
Train set size: 2300 samples (80.0%)
Test set size: 576 samples (20.0%)


In [54]:
print("\n" + "="*80)
print("DEFINING MODELS WITH HYPERPARAMETERS")
print("="*80)
models = {
    'LR': LogisticRegression(
        penalty='l2',
        solver='lbfgs',
        max_iter=1000,
        random_state=42,
        class_weight="balanced"
    ),
    'LightGBM': LGBMClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=-1,
        random_state=42,
        verbose=-1,
        is_unbalance=True
    )
}


print(f"Total models to evaluate: {len(models)}")
for name in models.keys():
    print(f"  - {name}")


DEFINING MODELS WITH HYPERPARAMETERS
Total models to evaluate: 2
  - LR
  - LightGBM


In [55]:
print("\n" + "="*80)
print("TRAINING AND EVALUATING MODELS")
print("="*80)

all_results = []
detailed_results = {}

for model_name, model in models.items():
    print(f"\n{'-'*80}")
    print(f"Training {model_name}...")
    print(f"{'-'*80}")
    
    # Evaluate on test set
    results, y_pred, y_pred_proba = evaluate_model(
        model, X_train, X_test, y_train, y_test, model_name
    )
    
    # Cross-validation on training set
    print(f"Performing 5-Fold Cross-Validation for {model_name}...")
    cv_results = cross_validate_model(model, X_train, y_train, cv=5)
    
    # Combine results
    results.update(cv_results)
    all_results.append(results)
    
    # Store detailed results
    detailed_results[model_name] = {
        'model': model,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'confusion_matrix': confusion_matrix(y_test, y_pred)
    }
    
    # Print summary
    print(f"Test Accuracy: {results['Accuracy']:.4f}")
    print(f"Test AUC: {results['AUC']:.4f}")
    print(f"CV Accuracy: {results['CV_Accuracy_mean']:.4f} (+/- {results['CV_Accuracy_std']:.4f})")



TRAINING AND EVALUATING MODELS

--------------------------------------------------------------------------------
Training LR...
--------------------------------------------------------------------------------
Performing 5-Fold Cross-Validation for LR...
Test Accuracy: 0.7274
Test AUC: 0.7780
CV Accuracy: 0.7087 (+/- 0.0090)

--------------------------------------------------------------------------------
Training LightGBM...
--------------------------------------------------------------------------------
Performing 5-Fold Cross-Validation for LightGBM...
Test Accuracy: 0.9462
Test AUC: 0.9315
CV Accuracy: 0.9543 (+/- 0.0051)


In [ ]:
from xgboost import XGBClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import classification_report

# 1. Khai báo model XGBoost cơ bản
xgb_base = XGBClassifier(
    objective="multi:softprob",
    num_class=2,          # nếu 3 lớp
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="mlogloss",
    random_state=42,
    n_jobs=-1,
)

# 2. RFE: chọn k features quan trọng nhất
n_features_to_select = 114   # mày chỉnh số này cho phù hợp

rfe = RFE(
    estimator=xgb_base,
    n_features_to_select=n_features_to_select,
    step=1
)

# 3. Fit RFE trên train
rfe.fit(X_train, y_train)

print("Số feature ban đầu:", X_train.shape[1])
print("Số feature sau khi chọn:", rfe.n_features_)

# 4. Lấy mask feature đã chọn
support_mask = rfe.support_          # mảng True/False
ranking = rfe.ranking_               # 1 = giữ lại, >1 = bị loại

# Nếu dùng pandas:
selected_features = X_train.columns[support_mask]
print("Các feature được chọn:")
print(selected_features)

# 5. Train model final trên tập feature đã chọn
pos = (y_train==1).sum()
neg = (y_train==0).sum()
scale_pos_weight = neg / pos
xgb_final = XGBClassifier(
    objective="multi:softprob",
    num_class=2,
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="mlogloss",
    random_state=42,
    n_jobs=-1,
    scale_pos_weight=scale_pos_weight
)

xgb_final.fit(X_train[selected_features], y_train)

# 6. Đánh giá
y_pred = xgb_final.predict(X_test[selected_features])
# print("y_test:", np.asarray(y_test).shape, type(y_test))
# print("y_pred:", np.asarray(y_pred).shape, type(y_pred))

y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred))


Số feature ban đầu: 114
Số feature sau khi chọn: 45
Các feature được chọn:
Index(['Z', 'u_flux_mean', 'u_flux_std', 'u_flux_max', 'u_flux_median',
       'u_amplitude', 'u_decay_time', 'u_auc', 'g_n_obs', 'g_flux_mean',
       'g_flux_std', 'g_flux_min', 'g_flux_max', 'g_flux_skew',
       'g_flux_kurtosis', 'g_decay_time', 'g_max_slope_up', 'g_auc',
       'r_flux_std', 'r_flux_min', 'r_flux_max', 'r_flux_median',
       'r_amplitude', 'r_flux_skew', 'r_flux_kurtosis', 'r_t_peak',
       'r_max_slope_up', 'r_max_slope_down', 'r_auc', 'i_flux_std',
       'i_flux_max', 'i_amplitude', 'i_flux_skew', 'i_decay_time', 'i_auc',
       'z_time_span', 'z_t_peak', 'z_max_slope_down', 'y_time_span',
       'y_flux_min', 'color_mean_r_i', 'color_mean_u_r', 'color_mean_g_i',
       'color_mean_r_z', 't_peak_diff_u_g'],
      dtype='object')
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       547
           1       0.50      0.14      0.22      

In [57]:
import numpy as np
print("y_test:", np.asarray(y_test).shape, type(y_test))
print("y_pred:", np.asarray(y_pred).shape, type(y_pred))


y_test: (576,) <class 'pandas.core.series.Series'>
y_pred: (576,) <class 'numpy.ndarray'>


In [ ]:
from tqdm import tqdm

class MLPWrapper:
    """Wrapper để MLPClassifier có thể dùng với RFE"""
    def __init__(self, hidden_layer_sizes=(100,), activation='relu', solver='adam', 
                 max_iter=1000, random_state=42, **kwargs):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation
        self.solver = solver
        self.max_iter = max_iter
        self.random_state = random_state
        self.mlp = MLPClassifier(
            hidden_layer_sizes=hidden_layer_sizes,
            activation=activation,
            solver=solver,
            max_iter=max_iter,
            random_state=random_state,
            **kwargs
        )
    
    def get_params(self, deep=True):
        """Trả về các tham số của estimator (cần cho sklearn clone)"""
        return {
            'hidden_layer_sizes': self.hidden_layer_sizes,
            'activation': self.activation,
            'solver': self.solver,
            'max_iter': self.max_iter,
            'random_state': self.random_state
        }
    
    def set_params(self, **params):
        """Thiết lập các tham số của estimator (cần cho sklearn clone)"""
        for key, value in params.items():
            setattr(self, key, value)
        # Tạo lại MLPClassifier với tham số mới
        self.mlp = MLPClassifier(
            hidden_layer_sizes=self.hidden_layer_sizes,
            activation=self.activation,
            solver=self.solver,
            max_iter=self.max_iter,
            random_state=self.random_state
        )
        return self
    
    def fit(self, X, y):
        self.mlp.fit(X, y)
        n_features = X.shape[1]
        
        # Tính feature importance từ coefs_ (trọng số của layer đầu tiên)
        if hasattr(self.mlp, 'coefs_') and len(self.mlp.coefs_) > 0:
            # coefs_[0] có shape (n_neurons, n_features)
            # Lấy giá trị tuyệt đối của trọng số và tính trung bình qua các neurons
            coefs_first_layer = self.mlp.coefs_[0]
            
            # Đảm bảo số lượng features khớp
            if coefs_first_layer.shape[1] == n_features:
                self.feature_importances_ = np.abs(coefs_first_layer).mean(axis=0)
            elif coefs_first_layer.shape[1] == n_features + 1:
                # Nếu có bias term, bỏ cột cuối cùng
                self.feature_importances_ = np.abs(coefs_first_layer[:, :-1]).mean(axis=0)
            else:
                # Nếu không khớp, dùng giá trị đều
                self.feature_importances_ = np.ones(n_features) / n_features
        else:
            # Nếu không có coefs_, dùng giá trị đều
            self.feature_importances_ = np.ones(n_features) / n_features
        
        # Đảm bảo feature_importances_ có đúng số lượng features
        if len(self.feature_importances_) != n_features:
            self.feature_importances_ = np.ones(n_features) / n_features
        
        return self
    
    def predict(self, X):
        return self.mlp.predict(X)
    
    def predict_proba(self, X):
        return self.mlp.predict_proba(X)
    
    def score(self, X, y):
        return self.mlp.score(X, y)

# Wrapper class để BaggingClassifier có thể dùng với RFE
class BaggingWrapper:
    """Wrapper để BaggingClassifier có thể dùng với RFE"""
    def __init__(self, estimator=None, n_estimators=10, random_state=42, **kwargs):
        self.estimator = estimator if estimator is not None else RandomForestClassifier()
        self.n_estimators = n_estimators
        self.random_state = random_state
        self.bagging = BaggingClassifier(
            estimator=self.estimator,
            n_estimators=n_estimators,
            random_state=random_state,
            **kwargs
        )
    
    def get_params(self, deep=True):
        """Trả về các tham số của estimator (cần cho sklearn clone)"""
        return {
            'estimator': self.estimator,
            'n_estimators': self.n_estimators,
            'random_state': self.random_state
        }
    
    def set_params(self, **params):
        """Thiết lập các tham số của estimator (cần cho sklearn clone)"""
        for key, value in params.items():
            setattr(self, key, value)
        # Tạo lại BaggingClassifier với tham số mới
        self.bagging = BaggingClassifier(
            estimator=self.estimator,
            n_estimators=self.n_estimators,
            random_state=self.random_state
        )
        return self
    
    def fit(self, X, y):
        self.bagging.fit(X, y)
        n_features = X.shape[1]
        
        # Tính feature importance từ các base estimators
        if hasattr(self.bagging, 'estimators_') and len(self.bagging.estimators_) > 0:
            importances = []
            for estimator in self.bagging.estimators_:
                if hasattr(estimator, 'feature_importances_'):
                    imp = estimator.feature_importances_
                    # Đảm bảo số lượng features khớp
                    if len(imp) == n_features:
                        importances.append(imp)
                elif hasattr(estimator, 'coef_'):
                    coef = estimator.coef_
                    # Xử lý coef_ có thể có shape khác nhau
                    if coef.ndim == 1:
                        imp = np.abs(coef)
                    elif coef.ndim == 2:
                        imp = np.abs(coef[0])
                    else:
                        imp = np.abs(coef).mean(axis=0)
                    
                    # Đảm bảo số lượng features khớp
                    if len(imp) == n_features:
                        importances.append(imp)
                    elif len(imp) == n_features + 1:
                        # Nếu có bias term, bỏ phần tử cuối
                        importances.append(imp[:-1])
            
            if importances:
                # Đảm bảo tất cả importances có cùng shape
                importances = [imp for imp in importances if len(imp) == n_features]
                if importances:
                    self.feature_importances_ = np.mean(importances, axis=0)
                else:
                    self.feature_importances_ = np.ones(n_features) / n_features
            else:
                self.feature_importances_ = np.ones(n_features) / n_features
        else:
            self.feature_importances_ = np.ones(n_features) / n_features
        
        # Đảm bảo feature_importances_ có đúng số lượng features
        if len(self.feature_importances_) != n_features:
            self.feature_importances_ = np.ones(n_features) / n_features
        
        return self
    
    def predict(self, X):
        return self.bagging.predict(X)
    
    def predict_proba(self, X):
        return self.bagging.predict_proba(X)
    
    def score(self, X, y):
        return self.bagging.score(X, y)

# Hàm tạo model instances mới cho RFE
def get_rfe_estimator(model_name):
    """Tạo estimator mới cho RFE"""
    
    if model_name == 'LR':
        return LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000, random_state=42, class_weight="balanced")
    elif model_name == 'LightGBM':
        return LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42, verbose=-1, scale_pos_weight=scale_pos_weight)
    else:
        raise ValueError(f"Unknown model: {model_name}")

# Hàm tạo model instances mới cho training
def get_model_instance(model_name):
    """Tạo instance mới của model để tránh dùng lại model đã được fit"""
    if model_name == 'LR':
        return LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000, random_state=42)
    elif model_name == 'LightGBM':
        return LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42, verbose=-1)
    else:
        raise ValueError(f"Unknown model: {model_name}")

# Định nghĩa các số lượng features cần test
feature_counts = [114]

# Lưu trữ kết quả
results_by_features = []

print("="*80)
print("CHẠY RFE VỚI SỐ LƯỢNG FEATURE SELECTION KHÁC NHAU (NHIỀU MODEL)")
print("="*80)

# Chuẩn bị dữ liệu
X_features = X_base_features.copy()
y = y_base.copy()

# Chia train-test một lần để đảm bảo tính nhất quán
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
    X_features, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

# Danh sách các model cần test
model_names = ['LR', 'LightGBM']

# Vòng lặp qua các số lượng features
for n_features in tqdm(feature_counts, desc="Processing feature counts"):
    if n_features > X_features.shape[1]:
        continue
    
    print(f"\n{'='*80}")
    print(f"Testing with {n_features} features")
    print(f"{'='*80}")
    
    # Chạy từng model
    for model_name in model_names:
        try:
            # Chạy RFE feature selection với estimator tương ứng
            rfe_estimator = get_rfe_estimator(model_name)
            
            rfe = RFE(
                estimator=rfe_estimator,
                n_features_to_select=n_features,
                step=1
            )
            
            # Fit selector trên training set
            rfe.fit(X_train_full, y_train_full)
            X_train_selected = X_train_full.loc[:, rfe.support_]
            X_test_selected = X_test_full.loc[:, rfe.support_]
            
            # Tạo model instance mới cho training
            model = get_model_instance(model_name)
            
            # Train và evaluate
            model.fit(X_train_selected, y_train_full)
            y_pred = model.predict(X_test_selected)
            accuracy = accuracy_score(y_test_full, y_pred)
            
            # Tính F1 score
            f1 = f1_score(y_test_full, y_pred, average='macro')
            
            # Tính AUC
            try:
                y_pred_proba = model.predict_proba(X_test_selected)[:, 1]
                auc = roc_auc_score(y_test_full, y_pred_proba, multi_class='ovr', average='macro')
            except:
                auc = np.nan
            
            results_by_features.append({
                'n_features': n_features,
                'model': model_name,
                'accuracy': accuracy,
                'f1': f1,
                'auc': auc
            })
            
            print(f"  ✓ {model_name}: Accuracy = {accuracy:.4f}, F1 = {f1:.4f}, AUC = {auc:.4f}")
            
        except Exception as e:
            print(f"  ⚠️ Error with {model_name} at {n_features} features: {str(e)}")
            continue

print("\n✅ Hoàn thành việc chạy tất cả các model với RFE!")

# Chuyển đổi kết quả thành DataFrame
results_df_features = pd.DataFrame(results_by_features)
print(f"\nTổng số kết quả: {len(results_df_features)}")
print(results_df_features.head(10))

CHẠY RFE VỚI SỐ LƯỢNG FEATURE SELECTION KHÁC NHAU (NHIỀU MODEL)


Processing feature counts:   0%|          | 0/1 [00:00<?, ?it/s]


Testing with 45 features
  ✓ LR: Accuracy = 0.9497, F1 = 0.4871, AUC = 0.7533


Processing feature counts: 100%|██████████| 1/1 [00:08<00:00,  8.48s/it]

  ✓ LightGBM: Accuracy = 0.9514, F1 = 0.5986, AUC = 0.9141

✅ Hoàn thành việc chạy tất cả các model với RFE!

Tổng số kết quả: 2
   n_features     model  accuracy        f1       auc
0          45        LR  0.949653  0.487088  0.753325
1          45  LightGBM  0.951389  0.598566  0.914140


In [ ]:
from tqdm import tqdm

class MLPWrapper:
    """Wrapper để MLPClassifier có thể dùng với RFE"""
    def __init__(self, hidden_layer_sizes=(100,), activation='relu', solver='adam', 
                 max_iter=1000, random_state=42, **kwargs):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation
        self.solver = solver
        self.max_iter = max_iter
        self.random_state = random_state
        self.mlp = MLPClassifier(
            hidden_layer_sizes=hidden_layer_sizes,
            activation=activation,
            solver=solver,
            max_iter=max_iter,
            random_state=random_state,
            **kwargs
        )
    
    def get_params(self, deep=True):
        """Trả về các tham số của estimator (cần cho sklearn clone)"""
        return {
            'hidden_layer_sizes': self.hidden_layer_sizes,
            'activation': self.activation,
            'solver': self.solver,
            'max_iter': self.max_iter,
            'random_state': self.random_state
        }
    
    def set_params(self, **params):
        """Thiết lập các tham số của estimator (cần cho sklearn clone)"""
        for key, value in params.items():
            setattr(self, key, value)
        # Tạo lại MLPClassifier với tham số mới
        self.mlp = MLPClassifier(
            hidden_layer_sizes=self.hidden_layer_sizes,
            activation=self.activation,
            solver=self.solver,
            max_iter=self.max_iter,
            random_state=self.random_state
        )
        return self
    
    def fit(self, X, y):
        self.mlp.fit(X, y)
        n_features = X.shape[1]
        
        # Tính feature importance từ coefs_ (trọng số của layer đầu tiên)
        if hasattr(self.mlp, 'coefs_') and len(self.mlp.coefs_) > 0:
            # coefs_[0] có shape (n_neurons, n_features)
            # Lấy giá trị tuyệt đối của trọng số và tính trung bình qua các neurons
            coefs_first_layer = self.mlp.coefs_[0]
            
            # Đảm bảo số lượng features khớp
            if coefs_first_layer.shape[1] == n_features:
                self.feature_importances_ = np.abs(coefs_first_layer).mean(axis=0)
            elif coefs_first_layer.shape[1] == n_features + 1:
                # Nếu có bias term, bỏ cột cuối cùng
                self.feature_importances_ = np.abs(coefs_first_layer[:, :-1]).mean(axis=0)
            else:
                # Nếu không khớp, dùng giá trị đều
                self.feature_importances_ = np.ones(n_features) / n_features
        else:
            # Nếu không có coefs_, dùng giá trị đều
            self.feature_importances_ = np.ones(n_features) / n_features
        
        # Đảm bảo feature_importances_ có đúng số lượng features
        if len(self.feature_importances_) != n_features:
            self.feature_importances_ = np.ones(n_features) / n_features
        
        return self
    
    def predict(self, X):
        return self.mlp.predict(X)
    
    def predict_proba(self, X):
        return self.mlp.predict_proba(X)
    
    def score(self, X, y):
        return self.mlp.score(X, y)

# Wrapper class để BaggingClassifier có thể dùng với RFE
class BaggingWrapper:
    """Wrapper để BaggingClassifier có thể dùng với RFE"""
    def __init__(self, estimator=None, n_estimators=10, random_state=42, **kwargs):
        self.estimator = estimator if estimator is not None else RandomForestClassifier()
        self.n_estimators = n_estimators
        self.random_state = random_state
        self.bagging = BaggingClassifier(
            estimator=self.estimator,
            n_estimators=n_estimators,
            random_state=random_state,
            **kwargs
        )
    
    def get_params(self, deep=True):
        """Trả về các tham số của estimator (cần cho sklearn clone)"""
        return {
            'estimator': self.estimator,
            'n_estimators': self.n_estimators,
            'random_state': self.random_state
        }
    
    def set_params(self, **params):
        """Thiết lập các tham số của estimator (cần cho sklearn clone)"""
        for key, value in params.items():
            setattr(self, key, value)
        # Tạo lại BaggingClassifier với tham số mới
        self.bagging = BaggingClassifier(
            estimator=self.estimator,
            n_estimators=self.n_estimators,
            random_state=self.random_state
        )
        return self
    
    def fit(self, X, y):
        self.bagging.fit(X, y)
        n_features = X.shape[1]
        
        # Tính feature importance từ các base estimators
        if hasattr(self.bagging, 'estimators_') and len(self.bagging.estimators_) > 0:
            importances = []
            for estimator in self.bagging.estimators_:
                if hasattr(estimator, 'feature_importances_'):
                    imp = estimator.feature_importances_
                    # Đảm bảo số lượng features khớp
                    if len(imp) == n_features:
                        importances.append(imp)
                elif hasattr(estimator, 'coef_'):
                    coef = estimator.coef_
                    # Xử lý coef_ có thể có shape khác nhau
                    if coef.ndim == 1:
                        imp = np.abs(coef)
                    elif coef.ndim == 2:
                        imp = np.abs(coef[0])
                    else:
                        imp = np.abs(coef).mean(axis=0)
                    
                    # Đảm bảo số lượng features khớp
                    if len(imp) == n_features:
                        importances.append(imp)
                    elif len(imp) == n_features + 1:
                        # Nếu có bias term, bỏ phần tử cuối
                        importances.append(imp[:-1])
            
            if importances:
                # Đảm bảo tất cả importances có cùng shape
                importances = [imp for imp in importances if len(imp) == n_features]
                if importances:
                    self.feature_importances_ = np.mean(importances, axis=0)
                else:
                    self.feature_importances_ = np.ones(n_features) / n_features
            else:
                self.feature_importances_ = np.ones(n_features) / n_features
        else:
            self.feature_importances_ = np.ones(n_features) / n_features
        
        # Đảm bảo feature_importances_ có đúng số lượng features
        if len(self.feature_importances_) != n_features:
            self.feature_importances_ = np.ones(n_features) / n_features
        
        return self
    
    def predict(self, X):
        return self.bagging.predict(X)
    
    def predict_proba(self, X):
        return self.bagging.predict_proba(X)
    
    def score(self, X, y):
        return self.bagging.score(X, y)

# Hàm tạo model instances mới cho RFE
def get_rfe_estimator(model_name):
    """Tạo estimator mới cho RFE"""
    
    if model_name == 'LR':
        return LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000, random_state=42, class_weight="balanced")
    elif model_name == 'LightGBM':
        return LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42, verbose=-1, scale_pos_weight=scale_pos_weight)
    else:
        raise ValueError(f"Unknown model: {model_name}")

# Hàm tạo model instances mới cho training
def get_model_instance(model_name):
    """Tạo instance mới của model để tránh dùng lại model đã được fit"""
    if model_name == 'LR':
        return LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000, random_state=42)
    elif model_name == 'LightGBM':
        return LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42, verbose=-1)
    else:
        raise ValueError(f"Unknown model: {model_name}")

# Định nghĩa các số lượng features cần test
feature_counts = [114]

# Lưu trữ kết quả
results_by_features = []

print("="*80)
print("CHẠY RFE VỚI SỐ LƯỢNG FEATURE SELECTION KHÁC NHAU (NHIỀU MODEL)")
print("="*80)

# Chuẩn bị dữ liệu
X_features = X_base_features.copy()
y = y_base.copy()

# Chia train-test một lần để đảm bảo tính nhất quán
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
    X_features, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

# Danh sách các model cần test
model_names = ['LR', 'LightGBM']

# Vòng lặp qua các số lượng features
for n_features in tqdm(feature_counts, desc="Processing feature counts"):
    if n_features > X_features.shape[1]:
        continue
    
    print(f"\n{'='*80}")
    print(f"Testing with {n_features} features")
    print(f"{'='*80}")
    
    # Chạy từng model
    for model_name in model_names:
        try:
            # Chạy RFE feature selection với estimator tương ứng
            rfe_estimator = get_rfe_estimator(model_name)
            
            rfe = RFE(
                estimator=rfe_estimator,
                n_features_to_select=n_features,
                step=1
            )
            
            # Fit selector trên training set
            rfe.fit(X_train_full, y_train_full)
            X_train_selected = X_train_full.loc[:, rfe.support_]
            X_test_selected = X_test_full.loc[:, rfe.support_]
            
            # Tạo model instance mới cho training
            model = get_model_instance(model_name)
            
            # Train và evaluate
            model.fit(X_train_selected, y_train_full)
            y_pred = model.predict(X_test_selected)
            accuracy = accuracy_score(y_test_full, y_pred)
            
            # Tính F1 score
            f1 = f1_score(y_test_full, y_pred, average='macro')
            
            # Tính AUC
            try:
                y_pred_proba = model.predict_proba(X_test_selected)[:, 1]
                auc = roc_auc_score(y_test_full, y_pred_proba, multi_class='ovr', average='macro')
            except:
                auc = np.nan
            
            results_by_features.append({
                'n_features': n_features,
                'model': model_name,
                'accuracy': accuracy,
                'f1': f1,
                'auc': auc
            })
            
            print(f"  ✓ {model_name}: Accuracy = {accuracy:.4f}, F1 = {f1:.4f}, AUC = {auc:.4f}")
            
        except Exception as e:
            print(f"  ⚠️ Error with {model_name} at {n_features} features: {str(e)}")
            continue

print("\n✅ Hoàn thành việc chạy tất cả các model với RFE!")

# Chuyển đổi kết quả thành DataFrame
results_df_features = pd.DataFrame(results_by_features)
print(f"\nTổng số kết quả: {len(results_df_features)}")
print(results_df_features.head(10))

CHẠY RFE VỚI SỐ LƯỢNG FEATURE SELECTION KHÁC NHAU (NHIỀU MODEL)


Processing feature counts:   0%|          | 0/1 [00:00<?, ?it/s]


Testing with 45 features
  ✓ LR: Accuracy = 0.9497, F1 = 0.4871, AUC = 0.7533


Processing feature counts: 100%|██████████| 1/1 [00:08<00:00,  8.68s/it]

  ✓ LightGBM: Accuracy = 0.9514, F1 = 0.5986, AUC = 0.9141

✅ Hoàn thành việc chạy tất cả các model với RFE!

Tổng số kết quả: 2
   n_features     model  accuracy        f1       auc
0          45        LR  0.949653  0.487088  0.753325
1          45  LightGBM  0.951389  0.598566  0.914140


In [61]:
print(model)

LGBMClassifier(random_state=42, verbose=-1)


In [62]:
print(models)

{'LR': LogisticRegression(class_weight='balanced', max_iter=1000, penalty='l2',
                   random_state=42), 'LightGBM': LGBMClassifier(is_unbalance=True, random_state=42, verbose=-1)}


# 📊 INFERENCE & PREDICTION

Phần này sẽ:
1. Load dữ liệu test
2. Preprocessing (giống training)
3. Dự đoán với model tốt nhất
4. Tạo file CSV submission


In [63]:
# ============================================================
# LOAD TEST DATA
# ============================================================

print("="*60)
print("LOADING TEST DATA")
print("="*60)

# Load test data
df_test = pd.read_csv('/home/duy/Downloads/Mallorn_update/Mallorn/mallorn-astronomical-classification-challenge/test_features_ml.csv')

print(f"Test data shape: {df_test.shape}")
print(f"\nFirst 3 rows:")
print(df_test.head(3))
print(f"\nColumns: {df_test.columns.tolist()[:10]}...")
print(f"\nChecking for NaN:")
print(f"Total NaN: {df_test.isnull().sum().sum()}")


LOADING TEST DATA
Test data shape: (7135, 116)

First 3 rows:
                  object_id       Z    Z_err    EBV  u_n_obs  u_time_span  \
0  Eluwaith_Mithrim_nothrim  0.5393  0.03013  0.610        2    1216.0901   
1        Eru_heledir_archam  0.7282  0.03508  0.058        9    1830.1833   
2         Gonhir_anann_fuin  0.6026  0.03185  0.070       17    2359.4595   

   u_flux_mean  u_flux_std  u_flux_min  u_flux_max  ...  color_mean_g_i  \
0     0.080995    0.063499    0.017497    0.144494  ...       -2.848042   
1     0.318856    0.438312   -0.472787    1.009501  ...       -0.941344   
2     0.154386    0.809561   -1.896650    1.650006  ...       -0.181608   

   color_mean_r_z  t_peak_diff_u_g  t_peak_diff_g_r  t_peak_diff_r_i  \
0       -2.747490         -28.9546        -636.9996         242.7726   
1       -0.350325         411.1488         -11.4208         702.3793   
2        0.047289        -212.9875        -652.7930         -38.7250   

   t_peak_diff_i_z  t_peak_diff_z_y  t_

In [64]:
# ============================================================
# PREPROCESS TEST DATA (SAME AS TRAINING)
# ============================================================

print("="*60)
print("PREPROCESSING TEST DATA")
print("="*60)
all_results = []
# # 1. Fill Z_err with 0
# print("1. Filling Z_err with 0...")
# if 'Z_err' in df_test.columns:
#     df_test['Z_err'] = df_test['Z_err'].fillna(0)
#     print(f"   ✓ Filled Z_err")

# 2. Drop rows with NaN in other columns
# print("\n2. Dropping rows with NaN in other columns...")
# rows_before = len(df_test)
# df_test_cleaned = df_test.dropna()
# rows_after = len(df_test_cleaned)
# print(f"   ✓ Dropped {rows_before - rows_after} rows ({(rows_before - rows_after)/rows_before*100:.2f}%)")


# 2. Điền MEDIAN vào các cột số còn lại
print("\n2. Điền MEDIAN vào các cột số còn lại...")
numerical_cols = df_test.select_dtypes(include=['float64', 'int64']).columns
numerical_cols = [col for col in numerical_cols if col not in ['Z_err', 'target']]

filled_count = 0
for col in numerical_cols:
    if df_test[col].isnull().sum() > 0:
        median_value = df_test[col].median()
        df_test[col] = df_test[col].fillna(median_value)
        filled_count += 1

print(f"   ✓ Đã điền MEDIAN vào {filled_count} cột")

# 3. Save object_id for submission
test_object_ids = df_test['object_id'].copy()
print(f"\n3. Saved {len(test_object_ids)} object IDs for submission")

# 4. Scale the features (using the SAME scaler from training)
print("\n4. Scaling features...")
df_test_scaled = df_test.copy()

# Get columns to scale (same as training)
columns_to_scale_test = [col for col in df_test_scaled.columns 
                          if col not in ['object_id', 'Z_err'] and col in columns_to_scale]

print(f"   Columns to scale: {len(columns_to_scale_test)}")

# Apply the SAME scaler from training data
df_test_scaled[columns_to_scale_test] = scaler.transform(df_test_scaled[columns_to_scale_test])
print(f"   ✓ Scaled using training scaler")

# 5. Prepare features (same as training)
print("\n5. Preparing features...")
exclude_cols_test = ['object_id', 'Z_err']
feature_cols_test = [col for col in df_test_scaled.columns 
                     if col not in exclude_cols_test 
                     and df_test_scaled[col].dtype in ['float64', 'int64']]

X_test_features = df_test_scaled[feature_cols_test].copy()

print(f"   ✓ Test features shape: {X_test_features.shape}")
print("\n" + "="*60)
print("✓ TEST DATA READY FOR INFERENCE!")
print("="*60)


# df_test = df_test.copy()

# # 1. Điền 0 vào cột Z_err
# print("1. Điền giá trị 0 vào cột Z_err...")
# df_test['Z_err'] = df_test['Z_err'].fillna(0)
# print(f"   ✓ Đã điền 0 vào {nan_counts['Z_err']} giá trị NaN trong Z_err")

# # 2. Điền MEDIAN vào các cột số còn lại
# print("\n2. Điền MEDIAN vào các cột số còn lại...")
# numerical_cols = df_test.select_dtypes(include=['float64', 'int64']).columns
# numerical_cols = [col for col in numerical_cols if col not in ['Z_err', 'target']]

# filled_count = 0
# for col in numerical_cols:
#     if df_test[col].isnull().sum() > 0:
#         median_value = df_test[col].median()
#         df_test[col] = df_test[col].fillna(median_value)
#         filled_count += 1

# print(f"   ✓ Đã điền MEDIAN vào {filled_count} cột")

# # 3. Kiểm tra xem còn NaN không
# print("\n3. Kiểm tra lại...")
# remaining_nan = df_test.isnull().sum().sum()
# if remaining_nan > 0:
#     print(f"   ⚠️ Còn {remaining_nan} NaN (có thể ở cột text)")
#     object_cols = df_test.select_dtypes(include=['object']).columns
#     for col in object_cols:
#         if col != 'object_id':
#             df_test[col] = df_test[col].fillna('Unknown')
#     print("   ✓ Đã điền 'Unknown' vào các cột text")
# else:
#     print("   ✓ Không còn NaN")

# print("\n" + "="*60)
# print("SAU KHI XỬ LÝ NaN")
# print("="*60)
# print(f"Kích thước DataFrame: {df_test.shape} (KHÔNG XÓA HÀNG ✓)")
# print("Số hàng đã xóa: 0")
# print(f"\nCòn NaN trong DataFrame không? {df_test.isnull().any().any()}")
# print(f"Tổng số NaN còn lại: {df_test.isnull().sum().sum()}")

# # Hiển thị thống kê cột Z_err sau khi điền
# print("\nThống kê cột Z_err sau khi điền 0:")
# print(f"  - Min: {df_test['Z_err'].min()}")
# print(f"  - Max: {df_test['Z_err'].max()}")
# print(f"  - Mean: {df_test['Z_err'].mean():.4f}")
# print(f"  - Số giá trị = 0: {(df_test['Z_err'] == 0).sum()}")

# # Cập nhật df với bản đã điền
# df = df_test.copy()

# print("\n✓ DataFrame đã được cập nhật!")
# print(f"✓ Kích thước hiện tại: {df.shape}")
# print(f"✓ Đã GIỮ NGUYÊN tất cả {len(df)} hàng dữ liệu!")

# X_test_features = df_test_scaled[feature_cols_test].copy()

# print(f"   ✓ Test features shape: {X_test_features.shape}")
# print("\n" + "="*60)
# print("✓ TEST DATA READY FOR INFERENCE!")
# print("="*60)


PREPROCESSING TEST DATA

2. Điền MEDIAN vào các cột số còn lại...
   ✓ Đã điền MEDIAN vào 91 cột

3. Saved 7135 object IDs for submission

4. Scaling features...
   Columns to scale: 114
   ✓ Scaled using training scaler

5. Preparing features...
   ✓ Test features shape: (7135, 114)

✓ TEST DATA READY FOR INFERENCE!


In [65]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve, f1_score

# ============================================================
# 1) Chọn đúng cột (đúng thứ tự) + check thiếu cột
# ============================================================
selected_features = list(selected_features)

missing = [c for c in selected_features if c not in X_test_features.columns]
if missing:
    raise ValueError(f"X_test_features thiếu các cột: {missing}")

X_kaggle_test_sel = X_test_features.loc[:, selected_features].copy()

# ============================================================
# 2) TÌM THRESHOLD TỐT NHẤT TRÊN VALIDATION (F1)
#    Bạn cần có (X_val_features, y_val) hoặc dùng holdout (X_test, y_test) của bạn.
#    Nếu notebook bạn đang gọi tập holdout là X_test/y_test thì thay bằng biến đó.
# ============================================================
def best_f1_threshold(y_true, proba1):
    precision, recall, thresholds = precision_recall_curve(y_true, proba1)
    # thresholds có length = len(precision)-1
    f1 = 2 * precision[:-1] * recall[:-1] / (precision[:-1] + recall[:-1] + 1e-12)
    best_idx = int(np.nanargmax(f1))
    return float(thresholds[best_idx]), float(f1[best_idx])

# Ví dụ: dùng holdout validation của bạn (đổi tên biến cho đúng notebook)
# X_val_sel = X_test.loc[:, selected_features].copy()
# proba_val = model.predict_proba(X_val_sel)[:, 1]
# thr, best_f1 = best_f1_threshold(y_test.values, proba_val)

# Nếu bạn chưa có validation riêng, tạm dùng threshold “nhẹ” để tránh all-0:
thr = 0.002  # bạn có thể thử 0.1, 0.05 nếu lớp 1 cực hiếm

print("Using threshold =", thr)
print(X_kaggle_test_sel.shape)
# ============================================================
# 3) Predict PROBA trên Kaggle test + áp threshold -> label
# ============================================================
# proba_test = model.predict_proba(X_test_features)[:, 1]
proba_test = model.predict_proba(X_kaggle_test_sel)[:, 1]

y_pred_test = (proba_test >= thr).astype(int)

# ============================================================
# 4) Create submission
# ============================================================
submission = pd.DataFrame({
    "object_id": test_object_ids,
    "prediction": y_pred_test
})

submission_path = "/home/duy/Downloads/Mallorn_update/Mallorn/submission.csv"
submission.to_csv(submission_path, index=False)

print(f"✓ Submission file created: {submission_path}")
print("Prediction distribution:")
print(submission["prediction"].value_counts())
print("proba1 min/mean/max:", proba_test.min(), proba_test.mean(), proba_test.max())


Using threshold = 0.002
(7135, 45)
✓ Submission file created: /home/duy/Downloads/Mallorn_update/Mallorn/submission.csv
Prediction distribution:
prediction
0    7126
1       9
Name: count, dtype: int64
proba1 min/mean/max: 8.54004514464648e-06 0.0001559921464739057 0.005262088976311692
